In [ ]:
import numpy as np # Imports the NumPy library, commonly used for numerical operations.
import matplotlib.pyplot as plt # Imports the pyplot module from Matplotlib, used for plotting.
import matplotlib.animation as animation # Imports the animation module for creating animations.
import os # Imports the os module, used for interacting with the operating system (like saving files).

# ---------------- PARAMETERS ---------------
nx, ny = 200, 100 # Defines the number of grid points in the x and y directions.
Lx, Ly = 2.0, 1.0 # Defines the physical dimensions of the simulation domain.
dx, dy = Lx / nx, Ly / ny # Calculates the grid spacing in the x and y directions.
c_s = 0.1 # Defines the sound speed of the ambient medium.
CFL = 0.4 # Defines the Courant-Friedrichs-Lewy number, a stability criterion for numerical simulations.
t_final = 3.0 # Defines the final simulation time.
output_interval = 0.05 # Defines the time interval between saving output frames.
jet_rho = 5.0 # Defines the density of the jet.
ambient_rho = 1.0 # Defines the density of the ambient medium.
jet_vx = 1.0 # Defines the initial velocity of the jet in the x-direction.
jet_radius = 0.08 * Ly # Defines the radius of the jet nozzle.
nozzle_center = 0.5 * Ly # Defines the y-coordinate of the jet nozzle center.
mu = 1.27 # Defines a parameter related to the H-alpha emissivity calculation.

# ---------------- GRID ---------------
x = (np.arange(nx) + 0.5) * dx # Creates an array of x-coordinates for the cell centers.
y = (np.arange(ny) + 0.5) * dy # Creates an array of y-coordinates for the cell centers.
X_img, Y_img = np.meshgrid(x, y, indexing='xy') # Creates 2D coordinate grids for plotting (for imshow).
X, Y = np.meshgrid(x, y, indexing='ij') # Creates 2D coordinate grids for calculations (for indexing).

# ---------------- INITIAL CONDITIONS ---------------
U = np.zeros((3, nx, ny)) # Initializes a 3D array U to store the conserved variables (density, momentum x, momentum y).
U[0] = ambient_rho # Sets the initial density to the ambient density.
U[1] = 0.0 # Sets the initial x-momentum to zero.
U[2] = 0.0 # Sets the initial y-momentum to zero.
tracer = np.zeros((nx, ny)) # Initializes a 2D array to track the jet material.

# Initial blob
r = np.sqrt((Y - nozzle_center)**2 + (X - 0.15 * Lx)**2) # Calculates the radial distance from a point.
mask_blob = r < 0.04 # Creates a boolean mask for a circular blob.
U[0][mask_blob] = jet_rho # Sets the density inside the blob to the jet density.
U[1][mask_blob] = jet_rho * 0.6 * jet_vx # Sets the x-momentum inside the blob.
tracer[mask_blob] = 1.0 # Sets the tracer value inside the blob to 1.0.

# ---------------- HYDRO FUNCTIONS ----------------
def pressure_iso(rho): return c_s**2 * rho # Defines a function to calculate the pressure based on density (isothermal equation of state).

def primitive_from_U(U): # Defines a function to convert conserved variables (U) to primitive variables (rho, ux, uy).
    rho = U[0] # Extracts density.
    ux = U[1] / rho # Calculates x-velocity.
    uy = U[2] / rho # Calculates y-velocity.
    return rho, ux, uy # Returns the primitive variables.

def flux_x(U): # Defines a function to calculate the x-direction flux of the conserved variables.
    rho, ux, uy = primitive_from_U(U) # Gets primitive variables.
    Fx = np.zeros_like(U) # Initializes a 3D array for the x-flux.
    Fx[0] = rho * ux # Calculates mass flux in x.
    Fx[1] = rho * ux**2 + pressure_iso(rho) # Calculates x-momentum flux in x.
    Fx[2] = rho * ux * uy # Calculates y-momentum flux in x.
    return Fx # Returns the x-flux.

def flux_y(U): # Defines a function to calculate the y-direction flux of the conserved variables.
    rho, ux, uy = primitive_from_U(U) # Gets primitive variables.
    Fy = np.zeros_like(U) # Initializes a 3D array for the y-flux.
    Fy[0] = rho * uy # Calculates mass flux in y.
    Fy[1] = rho * ux * uy # Calculates x-momentum flux in y.
    Fy[2] = rho * uy**2 + pressure_iso(rho) # Calculates y-momentum flux in y.
    return Fy # Returns the y-flux.

def max_wave_speed(U): # Defines a function to calculate the maximum wave speed in the fluid.
    rho, ux, uy = primitive_from_U(U) # Gets primitive variables.
    return np.sqrt(ux**2 + uy**2) + c_s # Returns the maximum wave speed (hydrodynamic velocity + sound speed).

def rusanov_flux(F_L, F_R, U_L, U_R, s_max): # Defines the Rusanov numerical flux function.
    return 0.5 * (F_L + F_R) - 0.5 * s_max * (U_R - U_L) # Calculates the numerical flux.

def apply_boundaries(U, tracer): # Defines a function to apply boundary conditions.
    y_centers = (np.arange(ny) + 0.5) * dy # Gets the y-coordinates of cell centers.
    nozzle_mask = np.abs(y_centers - nozzle_center) <= jet_radius # Creates a mask for the jet nozzle.
    i = 0 # Index for the inflow boundary (left side).
    U[0, i, nozzle_mask] = jet_rho # Sets density at the nozzle.
    U[1, i, nozzle_mask] = jet_rho * jet_vx # Sets x-momentum at the nozzle.
    U[2, i, nozzle_mask] = 0.0 # Sets y-momentum at the nozzle.
    tracer[i, nozzle_mask] = 1.0 # Sets tracer at the nozzle.
    # Ambient outside nozzle
    U[0, i, ~nozzle_mask] = ambient_rho # Sets ambient density outside the nozzle.
    U[1, i, ~nozzle_mask] = 0.0 # Sets ambient x-momentum outside the nozzle.
    U[2, i, ~nozzle_mask] = 0.0 # Sets ambient y-momentum outside the nozzle.
    tracer[i, ~nozzle_mask] = 0.0 # Sets tracer outside the nozzle.
    # Outflow
    U[:, -1, :] = U[:, -2, :] # Applies outflow boundary condition on the right (x-direction).
    U[:, :, 0] = U[:, :, 1] # Applies outflow boundary condition on the bottom (y-direction).
    U[:, :, -1] = U[:, :, -2] # Applies outflow boundary condition on the top (y-direction).
    tracer[-1, :] = tracer[-2, :] # Applies outflow boundary condition for tracer on the right.
    tracer[:, 0] = tracer[:, 1] # Applies outflow boundary condition for tracer on the bottom.
    tracer[:, -1] = tracer[:, -2] # Applies outflow boundary condition for tracer on the top.

def compute_dt(U): # Defines a function to compute the time step based on the CFL condition.
    s = max_wave_speed(U) # Gets the maximum wave speed across the grid.
    max_s = np.max(s) # Finds the global maximum wave speed.
    if max_s <= 0: # Handles the case of zero wave speed to avoid division by zero.
        return 1e-6 # Returns a small time step.
    dt_x = CFL * dx / max_s # Calculates time step based on x-direction spacing and wave speed.
    dt_y = CFL * dy / max_s # Calculates time step based on y-direction spacing and wave speed.
    return min(dt_x, dt_y) # Returns the minimum of the two time steps to ensure stability in both directions.

# ---------------- TIME LOOP ----------------
frames_rho, frames_Ha = [], [] # Initializes lists to store density and H-alpha emissivity frames for animation.
frames_ux, frames_uy = [], [] # Initializes lists to store x and y velocity frames.
frames_shock_mask = [] # Initializes a list to store shock mask frames.

t, step, next_output = 0.0, 0, 0.0 # Initializes time, step counter, and next output time.
max_steps = 12000 # Defines the maximum number of simulation steps.

# ---------------- TURBULENCE AND KNOT SETUP ----------------
turbulence_factor = 0.05 # Defines a factor for adding turbulence.
knot_active = False # Flag to track if a knot has formed.

while t < t_final and step < max_steps: # Main time loop: continues until final time or max steps are reached.
    dt = compute_dt(U) # Computes the time step for the current step.
    if t + dt > t_final: dt = t_final - t # Adjusts the time step to end exactly at t_final.

    apply_boundaries(U, tracer) # Applies boundary conditions to U and tracer.

    Fx = flux_x(U); Fy = flux_y(U) # Computes the fluxes in the x and y directions.
    U_left, U_right = U[:, :-1, :], U[:, 1:, :] # Slices U to get left and right states for x-direction flux calculation.
    Fxl, Fxr = Fx[:, :-1, :], Fx[:, 1:, :] # Slices Fx to get left and right fluxes for x-direction.
    smax_x = np.maximum(max_wave_speed(U_left), max_wave_speed(U_right)) # Calculates maximum wave speed between cells in x-direction.
    Fx_if = rusanov_flux(Fxl, Fxr, U_left, U_right, smax_x) # Computes the Rusanov flux at the interfaces in x.

    U_down, U_up = U[:, :, :-1], U[:, :, 1:] # Slices U to get down and up states for y-direction flux calculation.
    Fyd, Fyu = Fy[:, :, :-1], Fy[:, :, 1:] # Slices Fy to get down and up fluxes for y-direction.
    smax_y = np.maximum(max_wave_speed(U_down), max_wave_speed(U_up)) # Calculates maximum wave speed between cells in y-direction.
    Fy_if = rusanov_flux(Fyd, Fyu, U_down, U_up, smax_y) # Computes the Rusanov flux at the interfaces in y.

    U_new = U.copy() # Creates a copy of U to store the updated values.
    U_new[:, 1:-1, :] -= dt / dx * (Fx_if[:, 1:, :] - Fx_if[:, :-1, :]) # Updates U based on x-flux divergence (excluding boundaries).
    U_new[:, :, 1:-1] -= dt / dy * (Fy_if[:, :, 1:] - Fy_if[:, :, :-1]) # Updates U based on y-flux divergence (excluding boundaries).

    # ---------------- TURBULENCE ----------------
    tracer *= 0.995 # Decays the tracer value slightly to represent diffusion/mixing.
    rho_temp, ux_temp, uy_temp = primitive_from_U(U_new) # Gets primitive variables from the updated U.
    ux_temp += turbulence_factor * (np.random.rand(nx, ny) - 0.5) # Adds random noise to the x-velocity for turbulence.
    uy_temp += turbulence_factor * (np.random.rand(nx, ny) - 0.5) # Adds random noise to the y-velocity for turbulence.
    U_new[1] = rho_temp * ux_temp # Updates the x-momentum in U_new.
    U_new[2] = rho_temp * uy_temp # Updates the y-momentum in U_new.
    tracer[rho_temp > 1.5 * ambient_rho] = 1.0 # Sets tracer to 1.0 in regions of high density (potential shock fronts).

    # ---------------- KNOT FORMATION ----------------
    if (not knot_active) and (np.max(rho_temp) > 2.0 * jet_rho): # Checks if a knot should form (high density and knot not already active).
        print(f"Knot formed at t={t:.2f}") # Prints a message when a knot forms.
        knot_active = True # Sets the knot_active flag to True.
        max_idx = np.unravel_index(np.argmax(rho_temp), rho_temp.shape) # Finds the index of the maximum density.
        radius = 3 # Defines a radius for the knot region.
        x0, y0 = max_idx # Gets the x and y indices of the maximum density.
        for i_r in range(max(0, x0-radius), min(nx, x0+radius)): # Loops over a square region around the maximum density.
            for j_r in range(max(0, y0-radius), min(ny, y0+radius)):
                tracer[i_r,j_r] = 1.5 # Sets the tracer value to 1.5 in the knot region.

    U_new[0] = np.maximum(U_new[0], 1e-6) # Ensures the density remains positive.
    U = U_new # Updates U with the new values.
    t += dt # Increments the time.
    step += 1 # Increments the step counter.

    if t >= next_output or t >= t_final: # Checks if it's time to save an output frame.
        rho, ux, uy = primitive_from_U(U) # Gets primitive variables for output.
        Ha = (rho / mu)**2 * (0.2 + 0.8 * tracer) # Calculates H-alpha emissivity.
        dux_dx = np.gradient(ux, dx, axis=0) # Calculates the gradient of x-velocity in the x-direction.
        duy_dy = np.gradient(uy, dy, axis=1) # Calculates the gradient of y-velocity in the y-direction.
        div = dux_dx + duy_dy # Calculates the divergence of the velocity field.
        div_thresh = np.percentile(div, 5) # Calculates the 5th percentile of the divergence (to identify convergent flow).
        shock_mask = (div < div_thresh) & (rho > 1.5 * ambient_rho) # Creates a mask to identify shock fronts (convergent flow and high density).

        frames_rho.append(np.log10(rho.T + 1e-12)) # Appends the log10 of density (transposed) to the frames list.
        frames_Ha.append(np.log10(Ha.T + 1e-12)) # Appends the log10 of H-alpha emissivity (transposed) to the frames list.
        frames_ux.append(ux.T) # Appends the x-velocity (transposed) to the frames list.
        frames_uy.append(uy.T) # Appends the y-velocity (transposed) to the frames list.
        frames_shock_mask.append(shock_mask.T) # Appends the shock mask (transposed) to the frames list.
        next_output += output_interval # Updates the time for the next output.

print(f"Simulation done: collected {len(frames_rho)} frames (steps={step}).") # Prints a message when the simulation is finished.

# ---------------- ANIMATION ----------------
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5)) # Creates a figure and two subplots for the animation.
im1 = ax1.imshow(frames_rho[0], origin='lower', extent=[0,Lx,0,Ly], aspect='auto', cmap='inferno') # Displays the initial density frame.
im2 = ax2.imshow(frames_Ha[0], origin='lower', extent=[0,Lx,0,Ly], aspect='auto', cmap='magma') # Displays the initial H-alpha emissivity frame.
ax1.set_title("Log Density + Velocity & Shock Fronts") # Sets the title for the left subplot.
ax2.set_title("Log Hα Emissivity") # Sets the title for the right subplot.
for ax in (ax1, ax2): # Loops through both subplots.
    ax.set_xlabel("x"); ax.set_ylabel("y") # Sets the x and y labels for both subplots.

q_skip = (slice(None,None,6), slice(None,None,6)) # Defines a slicing pattern to skip some quiver arrows for clarity.
Xq = X_img[q_skip] # Selects x-coordinates for quiver plots using the skipping pattern.
Yq = Y_img[q_skip] # Selects y-coordinates for quiver plots using the skipping pattern.
qv = ax1.quiver(Xq, Yq, frames_ux[0][q_skip], frames_uy[0][q_skip],
                color='cyan', scale=10, width=0.003) # Creates a quiver plot (velocity arrows) for the initial frame.
shock_scatter = ax1.scatter([], [], s=8, c='white', marker='o', edgecolors='black', linewidths=0.3) # Initializes an empty scatter plot for shock fronts.
cbar1 = plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04) # Adds a color bar for the density plot.
cbar2 = plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04) # Adds a color bar for the H-alpha emissivity plot.
plt.tight_layout() # Adjusts subplot parameters for a tight layout.

def update(i): # Defines the update function for the animation.
    im1.set_data(frames_rho[i]) # Updates the density data for the current frame.
    im2.set_data(frames_Ha[i]) # Updates the H-alpha emissivity data for the current frame.
    qv.set_UVC(frames_ux[i][q_skip], frames_uy[i][q_skip]) # Updates the velocity data for the quiver plot.
    mask = frames_shock_mask[i] # Gets the shock mask for the current frame.
    if np.any(mask): # Checks if there are any shock points in the current frame.
        ys, xs = np.nonzero(mask) # Gets the indices of the shock points.
        x_coords = (xs + 0.5) * dx # Converts x indices to x coordinates.
        y_coords = (ys + 0.5) * dy # Converts y indices to y coordinates.
        shock_scatter.set_offsets(np.column_stack([x_coords, y_coords])) # Updates the positions of the shock points.
        shock_scatter.set_sizes(np.full_like(x_coords, 5)) # Sets the size of the shock points.
        shock_scatter.set_alpha(0.9) # Sets the transparency of the shock points.
    else: # If no shock points are present.
        shock_scatter.set_offsets(np.empty((0,2))) # Sets the scatter plot offsets to empty.
    fig.suptitle(f"YSO Jet Evolution (t={i*output_interval:.2f})") # Sets the main title of the figure with the current time.
    return [im1, im2, qv, shock_scatter] # Returns the updated artists for blitting.

ani = animation.FuncAnimation(fig, update, frames=len(frames_rho), interval=100, blit=True) # Creates the animation object.

fps = 10 # Defines the frames per second for the animation.
dpi = 150 # Defines the dots per inch for saving the animation.
try: # Tries to save the animation using ffmpeg.
    Writer = animation.writers['ffmpeg'] # Gets the ffmpeg writer.
    writer = Writer(fps=fps, metadata=dict(artist='YSO-sim'), bitrate=1800) # Creates an ffmpeg writer instance.
    outname = "jet_shock_animation.mp4" # Defines the output filename for the mp4.
    ani.save(outname, writer=writer, dpi=dpi) # Saves the animation as an mp4.
    print(f"✅ Saved animation as {outname} using ffmpeg.") # Prints a success message.
except: # If ffmpeg is not available, save as a GIF using Pillow.
    from matplotlib.animation import PillowWriter # Imports the PillowWriter.
    print("⚠ ffmpeg not available, saving GIF.") # Prints a warning message.
    pillow = PillowWriter(fps=fps) # Creates a PillowWriter instance.
    outname = "jet_shock_animation.gif" # Defines the output filename for the GIF.
    ani.save(outname, writer=pillow, dpi=dpi) # Saves the animation as a GIF.
    print(f"✅ Saved animation as {outname} using PillowWriter.") # Prints a success message.
plt.close() # Closes the plot figure to free up memory.

Knot formed at t=0.14
Simulation done: collected 61 frames (steps=862).
✅ Saved animation as jet_shock_animation.mp4 using ffmpeg.
